In [1]:
import pandas as pd

In [17]:
df = pd.read_csv('data.csv', decimal=',')
df["title"]= pd.to_datetime(df["title"], format="%d.%m.%Y")

df.loc[df['name'].str.contains("1 kk"), 'name'] = 'euribor_1_month'
df.loc[df['name'].str.contains("3 kk"), 'name'] = 'euribor_3_month'
df.loc[df['name'].str.contains("6 kk"), 'name'] = 'euribor_6_month'
df.loc[df['name'].str.contains("12 kk"), 'name'] = 'euribor_12_month'
df = df.drop(df[df['name'].str.contains("Eonia")].index)

df.dropna(inplace=True)

display(df)
display(df.info())

df.to_csv('euribor_rates.csv', index=False)

,title,name,value
0,2023-04-25,euribor_1_month,3.024
1,2023-04-25,euribor_3_month,3.268
2,2023-04-25,euribor_6_month,3.635
3,2023-04-25,euribor_12_month,3.900
4,2023-04-24,euribor_1_month,3.031
...,...,...,...
31189,1999-01-05,euribor_12_month,3.187
31191,1999-01-04,euribor_1_month,3.242
31192,1999-01-04,euribor_3_month,3.234
31193,1999-01-04,euribor_6_month,3.223


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24968 entries, 0 to 31194
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   24968 non-null  datetime64[ns]
 1   name    24968 non-null  object        
 2   value   24968 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 780.2+ KB


None